In [ ]:
# 本模块：相似词提取

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import re

In [2]:
table = pd.read_csv('newsvec.csv')
table = table.drop(columns=['Unnamed: 0'])
table

,0,1,2,3,4,5,6,7,8,9,...,3357,3358,3359,3360,3361,3362,3363,3364,3365,3366
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.008697,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.018087,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.051985,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.008745,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2220,0.000000,0.000000,0.0,0.0,0.0,0.0,0.008650,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2221,0.000000,0.000000,0.0,0.0,0.0,0.0,0.018087,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2222,0.008248,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.009313
2223,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [4]:
# 词向量降维
wordvec = np.zeros((table.shape[1], table.shape[0]))
for i in range(table.shape[1]):
    wordvec[i] = table.iloc[:][str(i)].values
pca = PCA(n_components=50)
wordvec = pca.fit_transform(wordvec)
wordvec

array([[ 0.08070019, -0.01120037, -0.07255311, ..., -0.00780146,
         0.01254902,  0.01952784],
       [ 0.05239343, -0.02358709,  0.07303247, ...,  0.01028694,
        -0.00110389, -0.01588288],
       [-0.00397422,  0.0045811 ,  0.00450638, ...,  0.00556442,
        -0.0038145 ,  0.00943345],
       ...,
       [-0.00298157, -0.0069307 , -0.03520782, ..., -0.0070057 ,
        -0.00668105,  0.00249252],
       [ 0.00624983, -0.00603873,  0.00744236, ...,  0.01003365,
         0.00499044, -0.00338131],
       [-0.0163218 , -0.00010694, -0.00240172, ...,  0.00300156,
         0.00298709, -0.00115532]])

In [ ]:
# 计算词与词之间的余弦相似度
def similar(words):
    ret = np.zeros((table.shape[1], table.shape[1]))
    for i in range(table.shape[1]):
        for j in range(table.shape[1]):
            x = words[i]
            y = words[j]
            cosine = np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
            ret[i][j] = cosine
    return ret

cos_sim_word = pd.DataFrame(similar(wordvec), index=range(table.shape[1]), columns=range(table.shape[1]))
cos_sim_word.to_csv('cos_sim_word.csv', sep=',')
cos_sim_word

In [28]:
# 根据相似度收集近义词
file = open('vocab.txt', mode='r')
totword = file.read()
file.close()
totword = totword.split(' ')
dic = {}
for i in range(table.shape[1]):
    sortword = np.argsort(cos_sim_word[i].values, kind='quicksort')
    dic[totword[i]] = totword[sortword[-2]]+','+totword[sortword[-3]]+','+totword[sortword[-4]]
file2 = open('synonym.txt', mode='w')
file2.write(str(dic))
file2.close()

In [5]:
# 保存
f = open('synonym.txt', mode='r')
content = re.split(':\ |,\ |\'', f.read()[1:-1])
f.close()
content = [item for item in filter(lambda x: x != '', content)]
content

['claim',
 'accuse,deny,tell',
 'story',
 'direct,cast,classic',
 'relay',
 'medal,medallist,gold',
 'attention',
 'indoor,medallist,pole',
 'tough',
 'lose,good,defeat',
 'spot',
 'number,enter,top',
 'help',
 'many,say,people',
 'direction',
 'discipline,economics,stability',
 'index',
 'manufacturing,industrial,economist',
 'democracy',
 'pupils,electoral,enthusiasm',
 'sensitive',
 'privacy,interact,interface',
 'rupees',
 'operating,earned,richest',
 'victory',
 'win,chance,second',
 'listings',
 'jailed,organisations,pledged',
 'stick',
 'dangerous,reveals,campaigning',
 'chosen',
 'secondary,adapted,relating',
 'close',
 'lead,past,point',
 'recorder',
 'revolution,lifestyle,terrestrial',
 'negative',
 'albeit,apparently,zero',
 'thoroughly',
 'unbeaten,replacing,gameplay',
 'minute',
 'penalty,corner,ball',
 'postal',
 'domain,domains,reluctant',
 'straight',
 'sixth,eighth,beat',
 'branch',
 'citizens,banking,deadline',
 'budget',
 'tax,fiscal,stability',
 'asylum',
 'immigrat